In [1]:
import pandas as pd
from datasets import load_dataset
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import DebertaV2TokenizerFast
import json
import os
from datetime import datetime
from pyabsa import AspectTermExtraction as ATEPC


No CUDA GPU found in your device


C:\Users\FX506\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\spacy\cli\_util.py:23: DeprecationWarning: Importing 'parser.split_arg_string' is deprecated, it will only be available in 'shell_completion' in Click 9.0.
  from click.parser import split_arg_string
C:\Users\FX506\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\weasel\util\config.py:8: DeprecationWarning: Importing 'parser.split_arg_string' is deprecated, it will only be available in 'shell_completion' in Click 9.0.
  from click.parser import split_arg_string


In [2]:

product_file_path = r"full-00000-of-00001.parquet"
review_file_path = r"All_Beauty.jsonl"

product_df = pd.read_parquet(os.path.join("data", product_file_path))
df = pd.read_json(os.path.join("data", review_file_path), lines=True)

In [3]:
# instead of 2020 , i want the user to choose the year
selected_year = int(input("Enter the year you want to filter by (default is 2020): ") or 2020)  
df = df[df['timestamp'].dt.year == selected_year]

In [4]:
df

,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase
0,5,Such a lovely scent but not overpowering.,This spray is really nice. It smells really go...,[],B00YQ6X8EO,B00YQ6X8EO,AGKHLEW2SOWHNMFQIJGBECAF7INQ,2020-05-05 14:08:48.923,0,True
1,4,Works great but smells a little weird.,"This product does what I need it to do, I just...",[],B081TJ8YS3,B081TJ8YS3,AGKHLEW2SOWHNMFQIJGBECAF7INQ,2020-05-04 18:10:55.070,1,True
2,5,Yes!,"Smells good, feels great!",[],B07PNNCSP9,B097R46CSY,AE74DYR3QUGVPZJ3P7RFWBGIX7XQ,2020-05-16 21:41:06.052,2,True
4,5,A+,Love it,[],B08BZ63GMJ,B08BZ63GMJ,AFQLNQNQYFWQZPJQZS6V3NZU4QBQ,2020-12-30 10:02:43.534,0,True
5,4,Pretty Color,The polish was quiet thick and did not apply s...,[{'small_image_url': 'https://images-na.ssl-im...,B00R8DXL44,B00R8DXL44,AGMJ3EMDVL6OWBJF7CA5RGJLXN5A,2020-08-27 22:30:08.138,0,True
...,...,...,...,...,...,...,...,...,...,...
701509,5,Lo recomiendo,Excelente resultado.,[],B01MDTVZTZ,B01MDTVZTZ,AF4HRHKZUDYAXOU3RQN3CQZ7A35A,2020-11-28 21:15:33.807,0,True
701511,1,"Wrong product, wrong description",It is only for cleaning. It is not for shavin...,[],B087FX6PPM,B087FX6PPM,AGY4XVENEBLOGFGUCFOAR4VJDE7A,2020-10-11 02:02:34.655,0,True
701514,5,Worth the buy !,Love !! The five packs worked with all my hair...,[{'small_image_url': 'https://images-na.ssl-im...,B07X3QMQS6,B07X3QMQS6,AFDPITZLCGMHQF7LUTDUA6EPNYXQ,2020-09-30 18:21:00.051,0,True
701522,1,I got a completely different bottle than this.,What I got was a completely different bottle t...,[],B00K21WP3M,B00K21WP3M,AGZIX3V6YHAG3JCUUUWY57FBO5OA,2020-04-09 18:02:48.241,0,True


In [5]:
# from ydata_profiling import ProfileReport

# ProfileReport(df, title='Reviews Dataset Report', minimal=True, progress_bar=False, samples=None, correlations=None, interactions=None, explorative=True, notebook={'iframe':{'height': '600px'}}, html={'style':{'primary_color': '#583101'}}, missing_diagrams={'heatmap': False, 'dendrogram': False}).to_notebook_iframe()


In [6]:

# ============================================================================
# MONTH SELECTION
# ============================================================================
print("="*60)
print("BEAUTY PRODUCT ANALYSIS - MONTH SELECTOR")
print("="*60)

while True:
    try:
        month_input = input("\nEnter the month number (1-12): ").strip()
        selected_month = int(month_input)
        if 1 <= selected_month <= 12:
            break
        else:
            print("Please enter a number between 1 and 12")
    except ValueError:
        print("Invalid input. Please enter a number.")

BEAUTY PRODUCT ANALYSIS - MONTH SELECTOR
Invalid input. Please enter a number.


In [9]:
df.columns

Index(['rating', 'title', 'text', 'images', 'asin', 'parent_asin', 'user_id',
       'timestamp', 'helpful_vote', 'verified_purchase'],
      dtype='object')

In [10]:
# drop  duplicates 
df = df.drop(columns=['images'])
df = df.drop_duplicates ( keep="first")
# keep verified reviews only
df = df[df['verified_purchase'] == True]

In [11]:


# ============================================================================
# SETUP AND PATCHING
# ============================================================================
# Apply the nuclear patch for DeBERTa tokenizer
original_getattr = DebertaV2TokenizerFast.__getattr__ if hasattr(DebertaV2TokenizerFast, '__getattr__') else None

def custom_getattr(self, name):
    defaults = {
        'bos_token': None,
        'eos_token': None,
        'split_special_tokens': False,
        'cls_token': '[CLS]',
        'sep_token': '[SEP]',
        'unk_token': '[UNK]',
        'pad_token': '[PAD]',
        'mask_token': '[MASK]',
    }
    
    if name.endswith('_id') and name[:-3] in defaults:
        token_name = name[:-3]
        token = defaults.get(token_name)
        if token and hasattr(self, 'convert_tokens_to_ids'):
            try:
                return self.convert_tokens_to_ids(token)
            except:
                return 0
        return 0
    
    if name in defaults:
        private_name = f'_{name}'
        if hasattr(self, private_name):
            return getattr(self, private_name)
        return defaults[name]
    
    if original_getattr:
        return original_getattr(self, name)
    
    raise AttributeError(f"'{type(self).__name__}' object has no attribute '{name}'")

DebertaV2TokenizerFast.__getattr__ = custom_getattr

# Initialize the aspect extractor
print("\nLoading ABSA model...")
checkpoint_path = os.path.join(
    'checkpoints', 
    'ATEPC_ENGLISH_CHECKPOINT', 
    'fast_lcf_atepc_English_cdw_apcacc_82.36_apcf1_81.89_atef1_75.43'
)
aspect_extractor = ATEPC.AspectExtractor(checkpoint_path, auto_device=True)
print("Model loaded successfully!")

# ============================================================================
# LOAD AND FILTER DATA
# ============================================================================
print("\nLoading dataset...")
if df['timestamp'].dtype == 'O' or not pd.api.types.is_datetime64_any_dtype(df['timestamp']):
    df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')

df_filtered = df[df['timestamp'].dt.month == selected_month].copy()
print(f'Filtered rows for month {selected_month}: {len(df_filtered)}')

# ============================================================================
# CSV SETUP FOR RESUMABLE PROCESSING
# ============================================================================
output_csv = os.path.join("absa_output" ,
                            f"{datetime(1900, selected_month, 1).strftime('%b')}"
                            f"{str(selected_year)[-2:]}", 
                            f"beauty_absa_results.csv")
processed_ids = set()

# Check if CSV already exists and load processed IDs
if os.path.exists(output_csv):
    print(f"\nFound existing results file: {output_csv}")
    existing_df = pd.read_csv(output_csv, dtype={'review_id': str})
    processed_ids = set(existing_df['review_id'].astype(str).unique())
    print(f"Resuming from {len(processed_ids)} already processed reviews")
    start_row = len(processed_ids)
else:
    print(f"\nCreating new results file: {output_csv}")
    start_row = 0

# ============================================================================
# PROCESS REVIEWS WITH ABSA
# ============================================================================
print(f"\nProcessing reviews with ABSA...\n")
results = []
total_to_process = len(df_filtered)

for idx, (row_idx, row) in enumerate(df_filtered.iterrows()):
    review_id = str(row_idx)
    
    # Skip if already processed
    if review_id in processed_ids:
        continue

    review_text = row['title'] + " " + row['text'] if not pd.isna(row['title']) and not pd.isna(row['text']) else row['text'] if not pd.isna(row['text']) else row['title'] if not pd.isna(row['title']) else ""
    
    # Skip empty reviews    
    if not review_text.strip():
        continue
    
    try:
        prediction = aspect_extractor.predict(
            review_text,
            save_result=False,
            print_result=False,
            ignore_error=True
        )
        
        aspects = []
        if prediction and isinstance(prediction, dict):
            if 'aspect' in prediction and 'sentiment' in prediction:
                for i, aspect in enumerate(prediction['aspect']):
                    sentiment = prediction['sentiment'][i] if i < len(prediction['sentiment']) else 'Neutral'
                    confidence = prediction.get('confidence', [1.0])[i] if i < len(prediction.get('confidence', [1.0])) else 1.0
                    aspects.append({
                        'term': aspect,
                        'sentiment': sentiment,
                        'confidence': confidence
                    })
        
        result_dict = {
            'review_id': review_id,
            'rating': row.get('rating', 0),
            'text': review_text[:500],  # Truncate text for CSV storage
            'timestamp': row.get('timestamp', ''),
            'user_id': row.get('user_id', ''),
            'parent_asin': row.get('parent_asin', ''),
            'aspects_json': json.dumps(aspects),
        }
        
        results.append(result_dict)
        
        if len(results) % 10 == 0:
            print(f"Progress: {start_row + len(results)}/{total_to_process} reviews")
            
    except Exception as e:
        print(f"Error processing review {review_id}: {str(e)[:50]}")
        result_dict = {
            'review_id': review_id,
            'rating': row.get('rating', 0),
            'text': review_text[:500],
            'timestamp': row.get('timestamp', ''),
            'user_id': row.get('user_id', ''),
            'parent_asin': row.get('parent_asin', ''),
            'aspects_json': '[]',
        }
        results.append(result_dict)

# ============================================================================
# SAVE TO CSV (APPEND MODE)
# ============================================================================
if results:
    results_df = pd.DataFrame(results)
    
    if os.path.exists(output_csv):
        results_df.to_csv(output_csv, mode='a', header=False, index=False, encoding='utf-8')
    else:
        results_df.to_csv(output_csv, index=False, encoding='utf-8')
    
    print(f"\n✓ Saved {len(results)} new reviews to {output_csv}")
else:
    print("\nNo new reviews to process")

# ============================================================================
# LOAD ALL RESULTS FOR ANALYSIS
# ============================================================================
all_results_df = pd.read_csv(output_csv, dtype={'review_id': str})
print(f"Total reviews in file: {len(all_results_df)}")

# ============================================================================
# GENERATE ANALYSIS SUMMARY
# ============================================================================
print("\n" + "="*60)
print("BEAUTY PRODUCT ANALYSIS SUMMARY")
print("="*60)

all_aspects = {}
sentiment_counts = {'Positive': 0, 'Negative': 0, 'Neutral': 0}

for _, row in all_results_df.iterrows():
    try:
        aspects = json.loads(row['aspects_json'])
        for aspect in aspects:
            term = aspect['term']
            sentiment = aspect['sentiment']
            
            if term not in all_aspects:
                all_aspects[term] = {'Positive': 0, 'Negative': 0, 'Neutral': 0, 'total': 0}
            
            all_aspects[term][sentiment] += 1
            all_aspects[term]['total'] += 1
            sentiment_counts[sentiment] += 1
    except:
        continue

print(f"\nTop 10 Most Mentioned Beauty Aspects:")
if all_aspects:
    sorted_aspects = sorted(all_aspects.items(), key=lambda x: x[1]['total'], reverse=True)[:10]
    for term, counts in sorted_aspects:
        if counts['total'] > 0:
            sentiment_score = (counts['Positive'] - counts['Negative']) / counts['total']
            print(f"  {term}: {counts['total']} mentions (Score: {sentiment_score:.2f})")
            print(f"    Positive: {counts['Positive']}, Negative: {counts['Negative']}, Neutral: {counts['Neutral']}")
else:
    print("  No aspects detected in the processed reviews.")

print(f"\nOverall Beauty Product Sentiment Distribution:")
print(f"  Positive: {sentiment_counts['Positive']}")
print(f"  Negative: {sentiment_counts['Negative']}")
print(f"  Neutral: {sentiment_counts['Neutral']}")

print("\n" + "="*60)
print(f"✓ Analysis complete! Results saved to {output_csv}")
print("="*60)


Loading ABSA model...
[2025-10-21 09:58:43] (2.4.3) Load aspect extractor from checkpoints\ATEPC_ENGLISH_CHECKPOINT\fast_lcf_atepc_English_cdw_apcacc_82.36_apcf1_81.89_atef1_75.43
[2025-10-21 09:58:43] (2.4.3) config: checkpoints\ATEPC_ENGLISH_CHECKPOINT\fast_lcf_atepc_English_cdw_apcacc_82.36_apcf1_81.89_atef1_75.43\fast_lcf_atepc.config
[2025-10-21 09:58:43] (2.4.3) state_dict: checkpoints\ATEPC_ENGLISH_CHECKPOINT\fast_lcf_atepc_English_cdw_apcacc_82.36_apcf1_81.89_atef1_75.43\fast_lcf_atepc.state_dict
[2025-10-21 09:58:43] (2.4.3) model: None
[2025-10-21 09:58:43] (2.4.3) tokenizer: checkpoints\ATEPC_ENGLISH_CHECKPOINT\fast_lcf_atepc_English_cdw_apcacc_82.36_apcf1_81.89_atef1_75.43\fast_lcf_atepc.tokenizer
[2025-10-21 09:58:43] (2.4.3) Set Model Device: cpu
[2025-10-21 09:58:43] (2.4.3) Device Name: Unknown


C:\Users\FX506\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\huggingface_hub\file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at microsoft/deberta-v3-base were not used when initializing DebertaV2Model: ['lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.LayerNorm.weight', 'lm_predictions.lm_head.dense.weight', 'mask_predictions.dense.bias', 'mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.bias', 'mask_predictions.classifier.bias', 'mask_predictions.classifier.weight', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.dense.weight']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on an

Model loaded successfully!

Loading dataset...
Filtered rows for month 11: 9083

Creating new results file: absa_output\Nov20\beauty_absa_results.csv

Processing reviews with ABSA...



C:\Users\FX506\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
c:\Users\FX506\Desktop\Yosr_in_Copenhaguen_25-26\project\PyABSA\pyabsa\tasks\AspectTermExtraction\prediction\aspect_extractor.py:649: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_new.cpp:256.)
  lcf_cdm_vec = torch.tensor(


Progress: 10/9083 reviews
Progress: 20/9083 reviews
Progress: 30/9083 reviews
Progress: 40/9083 reviews
Progress: 50/9083 reviews
Progress: 60/9083 reviews
Progress: 70/9083 reviews
Progress: 80/9083 reviews
Progress: 90/9083 reviews
Progress: 100/9083 reviews
Progress: 110/9083 reviews
Progress: 120/9083 reviews
Progress: 130/9083 reviews
Progress: 140/9083 reviews
Progress: 150/9083 reviews
Progress: 160/9083 reviews
Progress: 170/9083 reviews
Progress: 180/9083 reviews
Progress: 190/9083 reviews
Progress: 200/9083 reviews
Progress: 210/9083 reviews
Progress: 220/9083 reviews
Progress: 230/9083 reviews
Progress: 240/9083 reviews
Progress: 250/9083 reviews
Progress: 260/9083 reviews
Progress: 270/9083 reviews
Progress: 280/9083 reviews
Progress: 290/9083 reviews
Progress: 300/9083 reviews
Progress: 310/9083 reviews
Progress: 320/9083 reviews
Progress: 330/9083 reviews
Progress: 340/9083 reviews
Progress: 350/9083 reviews
Progress: 360/9083 reviews
Progress: 370/9083 reviews
Progress: 

OSError: Cannot save file into a non-existent directory: 'absa_output\Nov20'

In [13]:
results_save = results

In [14]:
if results:
    results_df = pd.DataFrame(results)
    
    if os.path.exists(output_csv):
        results_df.to_csv(output_csv, mode='a', header=False, index=False, encoding='utf-8')
    else:
        results_df.to_csv(output_csv, index=False, encoding='utf-8')
    
    print(f"\n✓ Saved {len(results)} new reviews to {output_csv}")
else:
    print("\nNo new reviews to process")


✓ Saved 9083 new reviews to absa_output\Nov20\beauty_absa_results.csv
